# CARG GPKG to Shapefile (Colab Minimal)
This notebook uses only open libraries (GeoPandas/Fiona/Shapely/pyproj/dbfread).

In [ ]:
# LOAD WORKSPACE: clone repo and set path
import os, shutil, sys
base_path = '/content'
repo_name = 'BD-CARG-2shape'
repo_url  = 'https://github.com/BaterHub/BD-CARG-2shape.git'
repo_path = os.path.join(base_path, repo_name)
# clean any previous clone
if os.path.exists(repo_path):
    shutil.rmtree(repo_path)
%cd {base_path}
!git clone {repo_url}
%cd {repo_name}
if repo_path not in sys.path: sys.path.append(repo_path)
print('Repo ready:', repo_path)

In [ ]:
# INSTALL DEPENDENCIES
!pip -q install geopandas fiona shapely pyproj dbfread

In [ ]:
# UPLOAD INPUT (.gpkg) AND domini.zip
from google.colab import files
import zipfile, time
workspace = '/content/workspace'
os.makedirs(workspace, exist_ok=True)
uploads = files.upload()  # select .gpkg and domini.zip
gpkg_path = ''
for fn in uploads:
    src = '/content/' + fn
    dst = os.path.join(workspace, fn)
    os.replace(src, dst)
    if fn.lower().endswith('.gpkg'):
        gpkg_path = dst
    if fn.lower() == 'domini.zip':
        with zipfile.ZipFile(dst) as z:
            z.extractall(os.path.join(workspace, 'domini'))
assert gpkg_path, 'Please upload a .gpkg file'
domini_dir = os.path.join(workspace, 'domini')
# fallback: use domains committed in repo if local 'domini' not uploaded
repo_domini = '/content/BD-CARG-2shape/domini'
if (not os.path.exists(domini_dir)) and os.path.exists(repo_domini):
    domini_dir = repo_domini
print('workspace:', workspace)
print('gpkg     :', gpkg_path)
print('domini   :', domini_dir, 'exists:', os.path.exists(domini_dir))

In [ ]:
# RUN CONVERSION
from openlibs.gpkg_pipeline import process_gpkg
out_dir, count = process_gpkg(gpkg_path, domini_dir=domini_dir)
print('Processed layers:', count)
print('Output dir      :', out_dir)

In [ ]:
# DOWNLOAD RESULTS
import shutil
from google.colab import files as colab_files
zip_base = '/content/output'
shutil.make_archive(zip_base, 'zip', out_dir)
colab_files.download(zip_base + '.zip')